In [1]:
import pandas as pd

In [2]:
financials = pd.read_excel("../Data/financials_data.xlsx")
gmp = pd.read_excel("../Data/GMP_data.xlsx")
listings = pd.read_excel("../Data/listings_data.xlsx") 

In [3]:
financials.head()

,Company,Opening Date,Issue Type,Issue Amount (Rs.cr.),Period Ended,Assets (Rs.cr.),Revenue (Rs.cr.),Profit After Tax (Rs.cr.),Net Worth (Rs.cr.),Reserves and Surplus (Rs.cr.),Total Borrowing (Rs.cr.)
0,Excelsoft Technologies IPO,"Nov 19, 2025",Mainboard,500.00,"Jun 30, 2025",478.34,60.28,6.01,375.95,274.25,37.82
1,Capillary Technologies IPO,"Nov 14, 2025",Mainboard,877.50,"Sep 30, 2025",892.33,362.56,1.03,509.38,149.34,88.94
2,Fujiyama Power Systems IPO,"Nov 13, 2025",Mainboard,828.00,"Jun 30, 2025",1243.88,597.79,67.59,464.34,436.33,432.83
3,Tenneco Clean Air IPO,"Nov 12, 2025",Mainboard,3600.00,"Jun 30, 2025",2918.77,1316.43,168.09,1250.38,1204.30,NaN
4,Mahamaya Lifesciences IPO,"Nov 11, 2025",SME,66.91,"Jun 30, 2025",218.87,84.04,4.10,53.50,35.73,57.72


In [4]:
gmp.head()

,IPO,Listing Date,IPO Size,Sub,GMP,Anchor
0,Shreeji Global FMCG SME (SHETHJI),45973,₹80.75 Cr,3.27x,₹01,1
1,"Groww IPO (GROWW,544603)",45973,₹6632.30 Cr,17.6x,₹52,1
2,"Lenskart Solutions IPO (LENSKART,544600)",45971,₹7278.02 Cr,28.27x,₹101,1
3,"Studds Accessories IPO (STUDDS,544599)",45968,₹455.49 Cr,73.25x,₹352,1
4,Safecure SME (544596),45967,₹29.07 Cr,1.81x,₹01,0


In [5]:
listings.head(10)

,Company Name,Listing Date,Issue Price (Rs),Listing Day - Close Price (Rs),Listing Day Gain / Loss (%),Current Price BSE,Current Price NSE,Gain / Loss (%)
0,Billionbrains Garage Ventures Ltd.,"Nov 12, 2025",100,NaN,0.00,NaN,NaN,0.00
1,Shreeji Global FMCG Ltd.,"Nov 12, 2025",125,NaN,0.00,NaN,NaN,0.00
2,Lenskart Solutions Ltd.,"Nov 10, 2025",402,403.10,0.27,415.45,415.35,3.35
3,Studds Accessories Ltd.,"Nov 07, 2025",585,560.30,-4.22,549.90,549.95,-6.00
4,Safecure Services Ltd.,"Nov 06, 2025",102,77.55,-23.97,66.55,NaN,-34.75
5,Orkla India Ltd.,"Nov 06, 2025",730,713.65,-2.24,686.25,686.65,-5.99
6,Game Changers Texfab Ltd.,"Nov 04, 2025",102,109.35,7.21,114.15,NaN,11.91
7,Jayesh Logistics Ltd.,"Nov 03, 2025",122,117.80,-3.44,NaN,119.60,-1.97
8,Midwest Ltd.,"Oct 24, 2025",1065,1140.50,7.09,1107.55,1106.00,4.00
9,Shlokka Dyes Ltd.,"Oct 17, 2025",91,85.50,-6.04,66.98,NaN,-26.40


In [6]:
financials.columns = financials.columns.str.lower().str.replace(" ","_")
gmp.columns = gmp.columns.str.lower().str.replace(" ","_")
listings.columns = listings.columns.str.lower().str.replace(" ","_")

In [7]:
financials.rename(columns={"company":"company_name"}, inplace=True)
gmp.rename(columns={"ipo":"company_name"}, inplace=True)

In [8]:
financials["company_name"] = financials["company_name"].apply(lambda x : x.replace("IPO","").strip())

In [9]:
import re

gmp['company_name'] = gmp['company_name'].apply(
    lambda x: re.sub(r'SME.*', '', x).strip()
)

gmp['company_name'] = gmp['company_name'].apply(
    lambda x: re.sub(r'IPO.*', '', x).strip()
)


In [10]:
listings["company_name"] = listings["company_name"].apply(lambda x : x.replace("Ltd.","").strip())

In [11]:
data = pd.merge(financials,listings, how="inner", on="company_name")
data = pd.merge(data, gmp, how="left", on="company_name")

In [12]:
data.shape

(704, 23)

In [13]:
data.columns

Index(['company_name', 'opening_date', 'issue_type', 'issue_amount_(rs.cr.)',
       'period_ended', 'assets_(rs.cr.)', 'revenue_(rs.cr.)',
       'profit_after_tax_(rs.cr.)', 'net_worth_(rs.cr.)',
       'reserves_and_surplus_(rs.cr.)', 'total_borrowing_(rs.cr.)',
       'listing_date_x', 'issue_price_(rs)', 'listing_day_-_close_price_(rs)',
       'listing_day_gain_/_loss_(%)', 'current_price_bse', 'current_price_nse',
       'gain_/_loss_(%)', 'listing_date_y', 'ipo_size', 'sub', 'gmp',
       'anchor'],
      dtype='object')

In [14]:
data.rename(columns={"listing_day_-_close_price_(rs)":"listing_day_close_price"}, inplace=True)

In [15]:
features = ['company_name', 'issue_amount_(rs.cr.)', 'assets_(rs.cr.)', 'revenue_(rs.cr.)',
       'profit_after_tax_(rs.cr.)', 'net_worth_(rs.cr.)',
       'reserves_and_surplus_(rs.cr.)', 'total_borrowing_(rs.cr.)', 'issue_price_(rs)', 'listing_day_close_price',
        'sub', 'gmp','anchor']

In [16]:
data[features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   company_name                   704 non-null    object 
 1   issue_amount_(rs.cr.)          704 non-null    float64
 2   assets_(rs.cr.)                703 non-null    float64
 3   revenue_(rs.cr.)               703 non-null    float64
 4   profit_after_tax_(rs.cr.)      703 non-null    float64
 5   net_worth_(rs.cr.)             696 non-null    float64
 6   reserves_and_surplus_(rs.cr.)  406 non-null    float64
 7   total_borrowing_(rs.cr.)       416 non-null    float64
 8   issue_price_(rs)               704 non-null    int64  
 9   listing_day_close_price        703 non-null    float64
 10  sub                            592 non-null    object 
 11  gmp                            592 non-null    object 
 12  anchor                         592 non-null    flo

In [17]:
data[features].head(1)

,company_name,issue_amount_(rs.cr.),assets_(rs.cr.),revenue_(rs.cr.),profit_after_tax_(rs.cr.),net_worth_(rs.cr.),reserves_and_surplus_(rs.cr.),total_borrowing_(rs.cr.),issue_price_(rs),listing_day_close_price,sub,gmp,anchor
0,Shreeji Global FMCG,80.75,128.76,251.18,9.2,38.76,22.8,29.55,125,NaN,3.27x,₹01,1.0


In [18]:
data["ipo_size"] = data["ipo_size"].str.extract(r'(\d*\.\d+|\d+)').astype(float)

In [19]:
data["sub"] = data["sub"].str.extract(r'(\d*\.\d+|\d+)').astype(float)

In [20]:
data["gmp"] = data["gmp"].str.extract(r'(\d*\.\d+|\d+)').astype(float)

In [21]:
data[["ipo_size", "sub", "gmp"]].isnull().sum()

ipo_size    112
sub         112
gmp         112
dtype: int64

In [22]:
data.isnull().sum()

company_name                       0
opening_date                       0
issue_type                         0
issue_amount_(rs.cr.)              0
period_ended                       0
assets_(rs.cr.)                    1
revenue_(rs.cr.)                   1
profit_after_tax_(rs.cr.)          1
net_worth_(rs.cr.)                 8
reserves_and_surplus_(rs.cr.)    298
total_borrowing_(rs.cr.)         288
listing_date_x                     0
issue_price_(rs)                   0
listing_day_close_price            1
listing_day_gain_/_loss_(%)        0
current_price_bse                304
current_price_nse                216
gain_/_loss_(%)                    0
listing_date_y                   112
ipo_size                         112
sub                              112
gmp                              112
anchor                           112
dtype: int64

In [23]:
data.drop(data[data["company_name"] == "Shreeji Global FMCG"].index, inplace=True)

In [24]:
data["listing_gains"] = data["listing_day_close_price"] > data["issue_price_(rs)"]

In [25]:
data

,company_name,opening_date,issue_type,issue_amount_(rs.cr.),period_ended,assets_(rs.cr.),revenue_(rs.cr.),profit_after_tax_(rs.cr.),net_worth_(rs.cr.),reserves_and_surplus_(rs.cr.),...,listing_day_gain_/_loss_(%),current_price_bse,current_price_nse,gain_/_loss_(%),listing_date_y,ipo_size,sub,gmp,anchor,listing_gains
1,Lenskart Solutions,"Oct 31, 2025",Mainboard,7278.02,"Jun 30, 2025",10845.68,1946.10,61.17,6176.87,5855.43,...,0.27,415.45,415.35,3.35,45971.0,7278.02,28.27,101.0,1.0,True
2,Studds Accessories,"Oct 30, 2025",Mainboard,455.49,"Jun 30, 2025",586.61,152.01,20.25,469.77,450.09,...,-4.22,549.90,549.95,-6.00,45968.0,455.49,73.25,352.0,1.0,False
3,Orkla India,"Oct 29, 2025",Mainboard,1667.54,"Jun 30, 2025",3158.20,605.38,78.92,1931.12,2523.56,...,-2.24,686.25,686.65,-5.99,45967.0,1667.54,48.74,662.0,1.0,False
4,Game Changers Texfab,"Oct 28, 2025",SME,52.08,"Jun 30, 2025",52.25,24.11,4.27,25.27,12.76,...,7.21,114.15,NaN,11.91,45965.0,52.08,1.17,1.0,1.0,True
5,Jayesh Logistics,"Oct 27, 2025",SME,27.17,"Jun 30, 2025",65.22,25.25,2.02,19.35,13.01,...,-3.44,NaN,119.60,-1.97,45964.0,27.17,65.59,92.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,Viaz Tyres,"Feb 16, 2023",SME,19.00,"Sep 30, 2022",34.06,23.79,1.53,10.98,0.00,...,9.76,NaN,79.00,27.42,44986.0,19.00,5.79,21.0,0.0,True
700,Agarwal Float Glass India,"Feb 10, 2023",SME,8.72,"Sep 30, 2022",21.77,23.69,2.68,5.95,3.93,...,4.88,NaN,53.00,26.19,NaN,NaN,NaN,NaN,NaN,True
701,Shera Energy,"Feb 07, 2023",SME,32.92,"Sep 30, 2022",338.33,339.54,4.28,70.55,0.00,...,18.07,NaN,134.05,135.18,44974.0,32.92,47.36,113.0,1.0,True
702,Eastern Logica Infoway,"Jan 05, 2023",SME,16.07,"Sep 30, 2022",125.62,305.04,1.33,36.03,33.85,...,26.00,212.00,NaN,-5.78,44943.0,16.07,1.74,21.0,0.0,True


In [26]:
features.remove("listing_day_close_price")

In [27]:
features.append("listing_gains")

In [28]:
data.loc[data["issue_type"] == "SME",features].to_csv("../Data/clean_data.csv", index=False)

In [29]:
data[features]

,company_name,issue_amount_(rs.cr.),assets_(rs.cr.),revenue_(rs.cr.),profit_after_tax_(rs.cr.),net_worth_(rs.cr.),reserves_and_surplus_(rs.cr.),total_borrowing_(rs.cr.),issue_price_(rs),sub,gmp,anchor,listing_gains
1,Lenskart Solutions,7278.02,10845.68,1946.10,61.17,6176.87,5855.43,335.48,402,28.27,101.0,1.0,True
2,Studds Accessories,455.49,586.61,152.01,20.25,469.77,450.09,2.91,585,73.25,352.0,1.0,False
3,Orkla India,1667.54,3158.20,605.38,78.92,1931.12,2523.56,2.33,730,48.74,662.0,1.0,False
4,Game Changers Texfab,52.08,52.25,24.11,4.27,25.27,12.76,9.88,102,1.17,1.0,1.0,True
5,Jayesh Logistics,27.17,65.22,25.25,2.02,19.35,13.01,29.65,122,65.59,92.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,Viaz Tyres,19.00,34.06,23.79,1.53,10.98,0.00,20.03,62,5.79,21.0,0.0,True
700,Agarwal Float Glass India,8.72,21.77,23.69,2.68,5.95,3.93,11.68,42,NaN,NaN,NaN,True
701,Shera Energy,32.92,338.33,339.54,4.28,70.55,0.00,114.38,57,47.36,113.0,1.0,True
702,Eastern Logica Infoway,16.07,125.62,305.04,1.33,36.03,33.85,56.95,225,1.74,21.0,0.0,True


In [30]:
data.loc[data["issue_type"] == "SME",features]

,company_name,issue_amount_(rs.cr.),assets_(rs.cr.),revenue_(rs.cr.),profit_after_tax_(rs.cr.),net_worth_(rs.cr.),reserves_and_surplus_(rs.cr.),total_borrowing_(rs.cr.),issue_price_(rs),sub,gmp,anchor,listing_gains
4,Game Changers Texfab,52.08,52.25,24.11,4.27,25.27,12.76,9.88,102,1.17,1.0,1.0,True
5,Jayesh Logistics,27.17,65.22,25.25,2.02,19.35,13.01,29.65,122,65.59,92.0,1.0,False
7,Sihora Industries,10.03,15.90,5.86,0.45,5.71,1.98,5.71,66,1.27,1.0,0.0,True
9,Mittal Sections,50.26,50.48,28.17,1.47,11.81,3.94,20.70,143,2.25,1.0,0.0,False
11,Greenleaf Envirotech,20.75,25.38,39.08,4.70,12.41,7.79,2.49,136,3.84,1.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,Viaz Tyres,19.00,34.06,23.79,1.53,10.98,0.00,20.03,62,5.79,21.0,0.0,True
700,Agarwal Float Glass India,8.72,21.77,23.69,2.68,5.95,3.93,11.68,42,NaN,NaN,NaN,True
701,Shera Energy,32.92,338.33,339.54,4.28,70.55,0.00,114.38,57,47.36,113.0,1.0,True
702,Eastern Logica Infoway,16.07,125.62,305.04,1.33,36.03,33.85,56.95,225,1.74,21.0,0.0,True


In [31]:
data.loc[data["issue_type"] == "SME",features]

,company_name,issue_amount_(rs.cr.),assets_(rs.cr.),revenue_(rs.cr.),profit_after_tax_(rs.cr.),net_worth_(rs.cr.),reserves_and_surplus_(rs.cr.),total_borrowing_(rs.cr.),issue_price_(rs),sub,gmp,anchor,listing_gains
4,Game Changers Texfab,52.08,52.25,24.11,4.27,25.27,12.76,9.88,102,1.17,1.0,1.0,True
5,Jayesh Logistics,27.17,65.22,25.25,2.02,19.35,13.01,29.65,122,65.59,92.0,1.0,False
7,Sihora Industries,10.03,15.90,5.86,0.45,5.71,1.98,5.71,66,1.27,1.0,0.0,True
9,Mittal Sections,50.26,50.48,28.17,1.47,11.81,3.94,20.70,143,2.25,1.0,0.0,False
11,Greenleaf Envirotech,20.75,25.38,39.08,4.70,12.41,7.79,2.49,136,3.84,1.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,Viaz Tyres,19.00,34.06,23.79,1.53,10.98,0.00,20.03,62,5.79,21.0,0.0,True
700,Agarwal Float Glass India,8.72,21.77,23.69,2.68,5.95,3.93,11.68,42,NaN,NaN,NaN,True
701,Shera Energy,32.92,338.33,339.54,4.28,70.55,0.00,114.38,57,47.36,113.0,1.0,True
702,Eastern Logica Infoway,16.07,125.62,305.04,1.33,36.03,33.85,56.95,225,1.74,21.0,0.0,True
